In [1]:
import torch
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image
import cv2
import numpy as np

import sys
sys.path.append('../object_detection')
from image_detector import crop_minAreaRect

In [2]:
names = ['Apple Braeburn',
 'Apple Golden 1',
 'Apple Golden 2',
 'Apple Golden 3',
 'Apple Granny Smith',
 'Apple Red 1',
 'Apple Red 2',
 'Apple Red 3',
 'Apple Red Delicious',
 'Apple Red Yellow',
 'Apricot',
 'Avocado',
 'Avocado ripe',
 'Banana',
 'Banana Red',
 'Cactus fruit',
 'Carambula',
 'Cherry',
 'Clementine',
 'Cocos',
 'Dates',
 'Garbage',
 'Granadilla',
 'Grape Pink',
 'Grape White',
 'Grape White 2',
 'Grapefruit Pink',
 'Grapefruit White',
 'Guava',
 'Huckleberry',
 'Kaki',
 'Kiwi',
 'Kumquats',
 'Lemon',
 'Lemon Meyer',
 'Limes',
 'Litchi',
 'Mandarine',
 'Mango',
 'Maracuja',
 'Nectarine',
 'Orange',
 'Papaya',
 'Passion Fruit',
 'Peach',
 'Peach Flat',
 'Pear',
 'Pear Abate',
 'Pear Monster',
 'Pear Williams',
 'Pepino',
 'Pineapple',
 'Pitahaya Red',
 'Plum',
 'Pomegranate',
 'Quince',
 'Raspberry',
 'Salak',
 'Strawberry',
 'Tamarillo',
 'Tangelo']

In [3]:
loader = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image):
    """load image, returns cuda tensor"""
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [4]:
model_path = "model_finetuning_v2.pth"
image_path = "../test_images/ban_app.jpg"
model = torch.load(model_path)

In [5]:
def find_objects(path):
    boundaries = [(np.array([10, 10, 10]), np.array([28, 255, 255])), #yellow
                  (np.array([28, 10, 10]), np.array([80, 255, 255])), #green
                  (np.array([0, 10, 10]), np.array([10, 255, 255]))] #red


    image = cv2.imread(path)
    bordersize = 50
    image = cv2.copyMakeBorder(image, top=bordersize, bottom=bordersize,
                                left=bordersize, right=bordersize,
                                borderType=cv2.BORDER_CONSTANT,
                                value=[255, 255, 255])
    image1 = image.copy()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    images = []

    for (lower, upper) in boundaries:
        mask = cv2.inRange(hsv, lower, upper)
        res = cv2.bitwise_and(image, image, mask=mask)

        im2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE,
                                                    cv2.CHAIN_APPROX_SIMPLE)
        index = []

        for i in range(len(contours)):
            if len(contours[i]) < 50:
                index.append(i)

        contours = np.delete(contours, index)

        for contour in contours:
            rect = cv2.minAreaRect(contour)

            box = cv2.boxPoints(rect)
            box = np.int0(box)
            startX, startY = max(box, key = lambda x: x[0])[0], min(box, key = lambda x: x[1])[1]
            cv2.drawContours(image1, [box], 0, (0, 0, 255), 2)
            
            # display the prediction
            img = crop_minAreaRect(image, rect)
            croped_image = image_loader(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
            outputs = model(croped_image)[0]
            outputs = (outputs.data).cpu().numpy()
            ind = outputs.argsort()[-5:][::-1]
            
            label = ""
            for i in ind:
                label += "  {}: {}".format(names[i], outputs[i]) + "\n"
            print(label)
        
            y, dy = startY, 15
            for line in label.split('\n'):
                y += dy
                cv2.putText(image1, line, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)

            img[np.where((img == [0, 0, 0]).all(axis=2))] = [255, 255, 255]
            images.append(img)
            
    cv2.imshow('image', image1)
    cv2.waitKey(0)
    
    return images

In [6]:
images = find_objects("../test_images/3.jpg")

# for i in range(len(images)):
#     cv2.imshow(f"{i}", images[i])
# cv2.waitKey(0)

[[ 57 206]
 [ 57  50]
 [209  50]
 [209 206]]
  Garbage: 12.43777847290039
  Grape White 2: 8.503541946411133
  Quince: 7.701147079467773
  Apple Golden 2: 7.6313982009887695
  Maracuja: 6.345088005065918

